# Installing Required Libraries

In [ ]:
!pip install -qU pandas fuzzywuzzy rouge sentence-transformers scikit-learn nltk

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from rouge import Rouge
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Coverage

In [ ]:
def evaluate_coverage(fmea_df):
    """
    Evaluates the FMEA dataframe for completeness and consistency.
    """
    # Check for missing values in critical columns
    missing_effects = fmea_df['Potential Effects'].isnull().sum()
    missing_causes = fmea_df['Potential Causes'].isnull().sum()
    missing_actions = fmea_df['Recommended Actions'].isnull().sum()
    missing_effects_attr = fmea_df['Effects_Attr'].isnull().sum()
    missing_causes_attr = fmea_df['Causes_Attr'].isnull().sum()
    missing_actions_attr = fmea_df['Recommended_Actions_Attr'].isnull().sum()

    # Total rows
    total_rows = len(fmea_df)

    # Completeness metrics
    completeness = {
        "Total Rows": total_rows,
        "Missing Effects": missing_effects,
        "Missing Causes": missing_causes,
        "Missing Actions": missing_actions,
        "Missing Effects Attribution": missing_effects_attr,
        "Missing Causes Attribution": missing_causes_attr,
        "Missing Actions Attribution": missing_actions_attr,
    }

    # Consistency check: Ensure no duplicate (Process, Failure Mode) pairs
    duplicate_pairs = fmea_df.duplicated(subset=['SMT Process', 'Failure Mode']).sum()

    # Consistency metrics
    consistency = {
        "Duplicate Pairs": duplicate_pairs,
    }

    # Print evaluation results
    print("FMEA Evaluation Results:")
    print("Completeness:")
    for key, value in completeness.items():
        print(f"  {key}: {value}")

    print("\nConsistency:")
    for key, value in consistency.items():
        print(f"  {key}: {value}")

    return {"completeness": completeness, "consistency": consistency}

In [ ]:
generated_fmea_df = pd.read_csv('/content/Final Generated FMEA.csv')
evaluation_results = evaluate_coverage(generated_fmea_df)

In [ ]:
ground_truth_df = pd.read_csv('/content/Ground Truth FMEA.csv')
ground_truth_df.head()

# Quantitative Metrics

In [ ]:
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")
rouge = Rouge()

In [ ]:
def normalize_text(text):
    return str(text).strip().lower()

In [ ]:
def calculate_generation_metrics(generated_text, ground_truth_text):
    """Calculate BLEU, ROUGE, and Semantic Similarity for generated text."""
    # BLEU Score with smoothing
    smoothing = SmoothingFunction().method1
    bleu_score = sentence_bleu([ground_truth_text.split()], generated_text.split(), smoothing_function=smoothing)

    # ROUGE Score
    rouge = Rouge()
    rouge_scores = rouge.get_scores(generated_text, ground_truth_text)[0]

    # Semantic Similarity
    embeddings = semantic_model.encode([generated_text, ground_truth_text])
    semantic_similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    return {
        "BLEU": bleu_score,
        "ROUGE-L": rouge_scores["rouge-l"]["f"],
        "Semantic Similarity": semantic_similarity
    }

In [ ]:
def evaluate_fmea_metrics(ground_truth_df, generated_fmea_df):
    metrics = {
        "generation": {
            "Potential Effects": [],
            "Potential Causes": [],
            "Recommended Actions": []
        }
    }

    for _, row in generated_fmea_df.iterrows():
        # Match ground truth row
        gt_row = ground_truth_df[
            (ground_truth_df["SMT Process"] == row["SMT Process"]) &
            (ground_truth_df["Failure Mode"] == row["Failure Mode"])
        ]

        if not gt_row.empty:

            # Generation Metrics
            for col in ["Potential Effects", "Potential Causes", "Recommended Actions"]:
                generated_text = normalize_text(row[col])
                ground_truth_text = normalize_text(gt_row.iloc[0][col])
                generation_metrics = calculate_generation_metrics(generated_text, ground_truth_text)
                metrics["generation"][col].append(generation_metrics)

    return metrics

In [ ]:
metrics = evaluate_fmea_metrics(ground_truth_df, generated_fmea_df)

# Print results
for col, col_metrics in metrics["generation"].items():
    print(f"Metrics for {col}:")
    avg_bleu = sum(m["BLEU"] for m in col_metrics) / len(col_metrics)
    avg_rouge = sum(m["ROUGE-L"] for m in col_metrics) / len(col_metrics)
    avg_semantic = sum(m["Semantic Similarity"] for m in col_metrics) / len(col_metrics)
    print(f"  Average BLEU: {avg_bleu:.2f}")
    print(f"  Average ROUGE-L: {avg_rouge:.2f}")
    print(f"  Average Semantic Similarity: {avg_semantic:.2f}")